In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report

# Cargar el dataset Iris
iris = load_iris()
X = iris.data  # Características
y = iris.target.reshape(-1, 1)  # Etiquetas en formato de columna

# One-hot encoding para las etiquetas (corrección del parámetro)
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

# Normalizar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.3, random_state=42)

# Inicialización de parámetros de la red neuronal
np.random.seed(42)
input_size = X_train.shape[1]  # Número de características
hidden_size = 10  # Número de neuronas en la capa oculta
output_size = y_train.shape[1]  # Número de clases
learning_rate = 0.1
epochs = 1000

# Pesos y sesgos
W1 = np.random.rand(input_size, hidden_size)
b1 = np.random.rand(hidden_size)
W2 = np.random.rand(hidden_size, output_size)
b2 = np.random.rand(output_size)

# Función sigmoide
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Derivada de la función sigmoide
def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))

# Función softmax para la salida
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Evitar overflow numérico
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Entrenamiento
for epoch in range(epochs):
    # Paso hacia adelante
    z1 = np.dot(X_train, W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = softmax(z2)

    # Pérdida (entropía cruzada categórica)
    loss = -np.mean(np.sum(y_train * np.log(a2 + 1e-8), axis=1))

    # Paso hacia atrás (retropropagación)
    d_a2 = a2 - y_train
    d_W2 = np.dot(a1.T, d_a2)
    d_b2 = np.sum(d_a2, axis=0)

    d_a1 = np.dot(d_a2, W2.T) * sigmoid_derivative(z1)
    d_W1 = np.dot(X_train.T, d_a1)
    d_b1 = np.sum(d_a1, axis=0)

    # Actualizar parámetros
    W1 -= learning_rate * d_W1
    b1 -= learning_rate * d_b1
    W2 -= learning_rate * d_W2
    b2 -= learning_rate * d_b2

    # Imprimir pérdida periódicamente
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Predicciones en el conjunto de prueba
z1_test = np.dot(X_test, W1) + b1
a1_test = sigmoid(z1_test)
z2_test = np.dot(a1_test, W2) + b2
a2_test = softmax(z2_test)

# Etiquetas predichas
y_pred = np.argmax(a2_test, axis=1)
y_true = np.argmax(y_test, axis=1)

# Reporte de clasificación
print("\nReporte de Clasificación:")
print(classification_report(y_true, y_pred, target_names=iris.target_names))


Epoch 0, Loss: 1.0989
Epoch 100, Loss: 0.0619
Epoch 200, Loss: 0.0543
Epoch 300, Loss: 0.0503
Epoch 400, Loss: 0.0455
Epoch 500, Loss: 0.0382
Epoch 600, Loss: 0.0270
Epoch 700, Loss: 0.0179
Epoch 800, Loss: 0.0126
Epoch 900, Loss: 0.0094

Reporte de Clasificación:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        19
  versicolor       1.00      1.00      1.00        13
   virginica       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

